# 1. 
Comme $M$ est une matrice symmétrique définie positive, ses valeurs propres seront toutes réelles et positives et égales à ses valeurs singulières. Les valeurs propres de $M$ respecteront $Mv= cv$ pour une valeur propre $c$ réelle positive. Si on cherche maintenant les valeurs propres de $M + \lambda² I$, on peut voir que
$$
\begin{align}
 (M + \lambda² I)v &= dv\\
  Mv + \lambda²v &= dv\\
  Mv &= dv - \lambda²v\\
  cv &= dv - \lambda²v\\
  d &= c + \lambda²
\end{align}
$$

Donc les valeurs propres (et singulières) de $M + \lambda² I$ sont celles de $M$ mais décalé de $\lambda²$. De cela, si le nombre de conditionnement de $M$ est 
$$ \kappa(M) = \sigma_\text{max}(M) / \sigma_\text{min}(M) $$

alors 
$$ \kappa(M + \lambda²) = (\sigma_\text{max}(M) + \lambda²) / (\sigma_\text{min}(M)+ \lambda²)$$

Or comme $\lambda² > 0$, cela implique que $ \kappa(M) > \kappa(M + \lambda²)$.

# 2. 

# 3. 

# 4. 

# 5. 

# 6. 

In [ ]:
using LinearAlgebra
using SuiteSparse
using SparseArrays
using Printf
using Test

In [ ]:
function test_least_square(f::Function, minimum_norm::Bool)
    A_1 = [1. 2. 3.; 4. 5. 6.; 7. 8. 9.; 5. 7. 9.]
    b_1 = ones(Float64, 4)
    
    x = f(sparse(A_1), b_1)
    x_baseline = A_1 \ b_1

    r = A_1 * x - b_1
    r_baseline = A_1 * x_baseline - b_1
    @test norm(r - r_baseline) ≈ 0.0 atol=1e-6   

    if minimum_norm
        @test norm(x - x_baseline) ≈ 0.0 atol=1e-6
    end
end

In [ ]:
function golub_riley(A::SparseMatrixCSC{Float64}, b::Vector{Float64}, λ::Float64=0.1, ϵ::Float64=1e-6)
    m, n = size(A)
    x_k = zeros(Float64, n)
    b_k = [b; zeros(n)]
    A_augmented = sparse([A; λ * I])
    QR_A_augmented = qr(A_augmented)

    while true
        Δx_k = QR_A_augmented \ b_k
        x_k += Δx_k
        b_k = [b - A * x_k; zeros(n)]
        if (norm(Δx_k) / norm(x_k)) ≤ ϵ
            break
        end
    end
    
    return x_k
end


In [ ]:
@test test_least_square(golub_riley, false)

# 7. 

# 8. 

# 9. 

# 10. 

# 11. 

# 12. 

# 13. 

In [ ]:
using LDLFactorizations

In [ ]:
function golub_riley_2(A::SparseMatrixCSC{Float64}, b::Vector{Float64}, λ::Float64=0.1, ϵ::Float64=1e-6)
    m, n = size(A)
    x_k = zeros(Float64, n)
    b_augmented_k = [b; zeros(n)]

    # We could pass an upper triangular matrix to ldl for more memory efficiency
    K = [I A; adjoint(A) (-(λ^2) * I)]
    println(size(K))
    LDLT = ldl(K)

    while true
        Δsol_k = LDLT \ b_augmented_k
        Δr_k = Δsol_k[1:m]
        Δx_k = Δsol_k[m+1:m+n]
        x_k += Δx_k
        b_augmented_k[1:m] = b - A * x_k
        if (norm(Δx_k) / norm(x_k)) ≤ ϵ
            break
        end
    end
    
    return x_k
end

In [ ]:
test_least_square(golub_riley_2, false)

# 14. 

# 15. 